In [1]:
import subprocess
import re
import os
import xml.etree.ElementTree as ET
import pandas as pd
import json
from jinja2 import Template
import datetime

In [2]:
print(f"\033[1;31;47m   _____                 _     ______            _       _ _             ")
print(f"\033[1;31;47m  / ____|               | |   |  ____|          | |     (_) |            ")
print(f"\033[1;31;47m | (___   ___ ___  _   _| |_  | |__  __  ___ __ | | ___  _| |_ ___ _ __  ")
print(f"\033[1;31;47m  \___ \ / __/ _ \| | | | __| |  __| \ \/ / '_ \| |/ _ \| | __/ _ \ '__| ")
print(f"\033[1;31;47m  ____) | (_| (_) | |_| | |_  | |____ >  <| |_) | | (_) | | ||  __/ |    ")
print(f"\033[1;31;47m |_____/ \___\___/ \__,_|\__| |______/_/\_\ .__/|_|\___/|_|\__\___|_|    ")
print(f"\033[1;31;47m                                          | |                            ")
print(f"\033[1;31;47m                                          |_|    v1.0 BY PEDRO ORLANDO   \033[0;m")

   _____                 _     ______            _       _ _             
  / ____|               | |   |  ____|          | |     (_) |            
 | (___   ___ ___  _   _| |_  | |__  __  ___ __ | | ___  _| |_ ___ _ __  
  \___ \ / __/ _ \| | | | __| |  __| \ \/ / '_ \| |/ _ \| | __/ _ \ '__| 
  ____) | (_| (_) | |_| | |_  | |____ >  <| |_) | | (_) | | ||  __/ |    
 |_____/ \___\___/ \__,_|\__| |______/_/\_\ .__/|_|\___/|_|\__\___|_|    
                                          | |                            
                                          |_|    v1.0 BY PEDRO ORLANDO   


In [3]:
start_time =datetime.datetime.now()
print(f"\033[1;37mScript started at: \033[1;32m {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

Script started at:  2023-03-30 20:10:15


In [4]:
dir_path = "/home/kali/Desktop/scans/"
print(f"\033[1;37mOutput data will be stored in: \033[1;32m {dir_path}")
os.chdir(dir_path)

Output data will be stored in:  /home/kali/Desktop/scans/


In [5]:
def get_mi_ip():
    output = subprocess.check_output(['ifconfig', 'eth0']).decode()
    match = re.search(r'inet\s+(?P<ip>\d+\.\d+\.\d+\.\d+)', output)
    if match:
        private_ip = match.group('ip')
        return private_ip
    else:
        print("\033[1;31mPrivate IP address not found for eth0")

In [6]:
def get_network():
    oct1,oct2,oct3,oct4 = get_mi_ip().split(".")
    network_to_scan =f'{oct1}.{oct2}.{oct3}.0/24'
    print(f"\033[1;37mNetwork Range found: \033[1;32m {network_to_scan}")
    return network_to_scan

In [7]:
def get_hosts():
    up_hosts = []
    tree = ET.parse('hosts.xml')
    root = tree.getroot()
    for address in root.findall('.//address'):
        addr = address.get('addr')
        if addr != get_mi_ip():
            if address.get('addrtype') == 'ipv4':
                up_hosts.append(addr)
    return up_hosts   

In [8]:
def create_search(product, version = None):
    if product != None and version != None:
        return product + ' ' + version
    elif product != None and version == None:
        return product
    elif product == None and version != None:
        return version
    else:
        return None

In [9]:
def search_exploits(search_string):
    
    if search_string is not None:
        if search_string.isdigit() and len(search_string) == 1:
            search_string = None
    cmd = f"searchsploit -j {search_string}"
    exploits = subprocess.check_output(cmd, shell=True)
    output1 = exploits.decode('utf-8')
    to_json = json.loads(output1)
    
    return to_json
   


In [12]:
hosters = []
protocols = []
portids = []
ports_status = []
names = []
versions = []
exploits_name = []
urls= []
paths = []


In [13]:
hosts_request = f"nmap -sn -n -oX hosts.xml {get_network()}"
host_discovery = subprocess.check_output(hosts_request, shell=True)


Network Range found:  192.168.50.0/24


In [14]:
hosts = get_hosts()
found_hosts = ""
for host in hosts:
    found_hosts += host + "\n"
print(f"""\033[1;37mHosts up in the network: \033[1;32m {found_hosts}""")

Hosts up in the network:  192.168.50.137



In [15]:

for host in hosts:
    counter = 0
    service_request = f"nmap -sV -n -oX {host}.xml {host}"
    service_discovery = subprocess.check_output(service_request, shell=True)
    tree = ET.parse(f'{host}.xml')
    root = tree.getroot()
    print(f"\033[1;37mPort scan ended for host: \033[1;32m{host}")
    print(f"\033[1;33m File data created in the export directory")

    print(f"\033[1;37m Searching for \033[1;34m EXPLOITS")
    for port in root.findall('.//port'):
        
        protocol= port.get('protocol')
        portid = port.get('portid')
        state = port.find('state')
        port_status = state.get('state')
        service = port.find('service')

        if service is not None:
            name= service.get('name')
            product = service.get('product')
            version = service.get('version')
            
            if version is not int:
                versions.append(create_search(product, version))
            else:
                versions.append(create_search(product))
            events = search_exploits(create_search(product, version))
            
            for event in events['RESULTS_EXPLOIT']:
                hosters.append(host)
                protocols.append(protocol)
                portids.append(portid)
                ports_status.append(port_status)
                names.append(name)
                exploits_name.append(event['Title'])
                urls.append(event['Application'])
                paths.append(event['Path'])
                counter += 1
    
    print(f"\033[1;37m Found: \033[1;34m {counter} EXPLOITS")
            
            

Port scan ended for host: 192.168.50.137
 File data created in the export directory
 Searching for  EXPLOITS
 Found:  241 EXPLOITS


In [31]:
urls2 =[]
for i in urls:
    
    urls2.append(i.replace(".com", ".com/"))

In [ ]:
urls2

In [28]:
df = pd.DataFrame({
    'Host': hosters,
    'Protocol': protocols,
    'Port ID': portids,
    'Port Status': ports_status,
    "Exploits": exploits_name,
    "URL" : urls,
    'Paths': paths

})

In [27]:
print(f"\033[1;36mEXPORTING DATA...")

EXPORTING DATA...


In [29]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
pd.set_option('display.unicode.ambiguous_as_wide', True)

In [30]:
template = Template(open("plantilla.html").read())

html = template.render(title="Results", table=df.to_html(index=False, col_space=None))


with open("Result.html", "w") as f:
    f.write(html.replace("&lt;br&gt;", "<br>"))
    

In [31]:
hosts_quantity = len(hosts)

In [48]:
end_time = datetime.datetime.now()


In [68]:
print(f"\033[1;37mScripts ending: \033[1;32m{end_time.strftime('%Y-%m-%d %H:%M:%S')}")
duration_time = (end_time - start_time)
hosts_quantity = len(hosts)
exploit_quantity= len(exploits_name)
if hosts_quantity > 1:
    print(f"\033[1;32m{hosts_quantity} \033[1;37mhosts scannedScann time: \033[1;32m{str(duration_time).split('.')[0]}")
else:
    print(f"\033[1;32m{hosts_quantity} \033[1;37mhost scanned. Scann time: \033[1;32m{str(duration_time).split('.')[0]}")

if exploit_quantity > 1:
    print(f"\033[1;31m{exploit_quantity} \033[1;34mEXPLOITS \033[1;37mFOUNDED")
else:
    print(f"{exploit_quantity} EXPLOIT FOUNDED")

Scripts ending: 2023-03-30 16:23:53
1 host scanned. Scann time: 0:00:49
2410 EXPLOITS FOUNDED
